In [44]:
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam
from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import keras

In [38]:
batch_size = 64
num_channels = 1
num_classes = 20
image_size = 20
latent_dim = 400

In [29]:
# # We'll use all the available examples from both the training and test
# # sets.
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# all_digits = np.concatenate([x_train, x_test])
# all_labels = np.concatenate([y_train, y_test])

# # Scale the pixel values to [0, 1] range, add a channel dimension to
# # the images, and one-hot encode the labels.
# all_digits = all_digits.astype("float32") / 255.0
# all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
# all_labels = keras.utils.to_categorical(all_labels, 10)

# # Create tf.data.Dataset.
# dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
# dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

# print(f"Shape of training images: {all_digits.shape}")
# print(f"Shape of training labels: {all_labels.shape}")

In [30]:
# Dataset generator - generating symmetric matrices
dimension_2D=20
n_samples=20
start=0
end=20

# Extracting the distance matrix
f1=open('Dataset1.txt','r')
l1=[]
l1=[line.split() for line in f1]
distance=[[] for i in range(len(l1)//20)]
i1=0
j1=0
while True:
    distance[i1].append(list(float(x) for x in l1[j1]))
    if((j1+1)%20==0):
        i1+=1
    if(j1==len(l1)-1):
        break
    j1+=1

# Extracting the adjency matrix without distance
f2=open('Dataset2.txt','r')
l2=[]
l2=[line.split() for line in f2]
network=[[] for i in range(len(l2)//20)]
i2=0
j2=0
while True:
    network[i2].append(list(float(x) for x in l2[j2]))
    if((j2+1)%20==0):
        i2+=1
    if(j2==len(l2)-1):
        break
    j2+=1

distance=np.array(distance)
network=np.array(network)
network = network.astype('float32')
network = np.expand_dims(network, axis=-1)
distance = np.expand_dims(distance,axis= -1)
distance = distance.astype('float32')

# Extracting the number of nodes and adding as labels
# Data distance is a 1000*2 matrix which contains 1000 20*20*1 matrices
f3=open('Dataset_Labels.txt','r')
labels=[l.split() for l in f3]

# Adding labels to the distance matrix
data_distance = []
data_distance_labels = []
for i in range(1000):
    data_distance.append((distance[i]))
    data_distance_labels.append((int(labels[i][0])))
data_distance = np.array(data_distance,dtype=object)

# Adding labels to network matrix
# Data network is a 1000*2 matrix which contains 1000 20*20*1 matrices

data_network_train = []
data_network_labels_train=[]
for i in range(800):
    data_network_train.append(network[i])
    data_network_labels_train.append((int(labels[i][0])))
data_network_train = np.array(data_network_train)
data_network_train = data_network_train.astype('float32')
data_network_labels_train = np.array(data_network_labels_train)
data_network_labels_train = data_network_labels_train.astype('float32')

data_network_test = []
data_network_labels_test=[]
for i in range(800,1000):
    data_network_test.append(network[i])
    data_network_labels_test.append((int(labels[i][0])))
data_network_test = np.array(data_network_test)
data_network_test = data_network_test.astype('float32')
data_network_labels_test = np.array(data_network_labels_test)
data_network_labels_test = data_network_labels_test.astype('float32')

In [39]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

420 21


In [40]:
# Create the discriminator.
in_shape=(dimension_2D,dimension_2D,1)
discriminator = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=in_shape),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 5x5x(128 + num_classes) map.
        layers.Dense(5 * 5 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((5, 5, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (5, 5), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [41]:

class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }


In [46]:
data_network_labels_train = keras.utils.to_categorical(data_network_labels_train, num_classes)
data_network_labels_test = keras.utils.to_categorical(data_network_labels_test, num_classes)

In [47]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

# fit the model
model = cond_gan.fit(data_network_train, data_network_labels_train,batch_size=batch_size,epochs= 20,verbose=1,validation_data=(data_network_test, data_network_labels_test))

2022-12-05 21:15:18.753348: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 537600000 exceeds 10% of free system memory.


Epoch 1/20


ValueError: in user code:

    File "/home/jishnu/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/jishnu/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jishnu/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/tmp/ipykernel_13370/1953686470.py", line 38, in train_step
        random_vector_labels = tf.concat(

    ValueError: Shape must be rank 2 but is rank 5 for '{{node concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](random_normal, IteratorGetNext:1, concat/axis)' with input shapes: [?,400], [?,21,20,20,20], [].
